In [106]:
 # Dependencies
import requests
import json
import pandas as pd
from config import geoapify_key
from splinter import Browser
from bs4 import BeautifulSoup as soup
import re

In [107]:
browser = Browser('chrome')

In [108]:
url = 'https://www.expedia.com/Hotel-Search?HotelID=3016212&adults=2&children=&destination=New%20York%2C%20New%20York%2C%20United%20States%20of%20America&endDate=2023-07-01&gclid=CjwKCAjwpayjBhAnEiwA-7ena7JGaI1rwCkOUycQoEp0ICRA_jkQo7Qt2Wwm4iOrBE-_4GP68HEzBRoC6xIQAvD_BwE&latLong=&locale=en_US&mapBounds=&pwaDialog=&regionId=2621&semcid=US.UB.GOOGLE.PT-c-EN.HOTEL&semdtl=a115093754954.b1127871358663.g1kwd-42777028833.l1.e1c.m1CjwKCAjwpayjBhAnEiwA-7ena7JGaI1rwCkOUycQoEp0ICRA_jkQo7Qt2Wwm4iOrBE-_4GP68HEzBRoC6xIQAvD_BwE.r14d4f421864e74f5bf75be17391f1a3135e350e186a47a1456f6c4f1617a2988c.c10RNuppwrQ5tvtmka9Z9sWg.j19032180.k1.d1558118514604.h1e.i1.n1.o1.p1.q1.s1.t1.x1.f1.u1.v1.w1&siteid=1&sort=PRICE_LOW_TO_HIGH&startDate=2023-06-30&theme=&useRewards=false&userIntent='
browser.visit(url)

In [109]:
# Set the geographical coordinates for NYC
latitude = 40.7128
longitude = -74.0060

# Set the parameters for the type of place
categories = "accommodation"

# Set the parameters for the type of search
bias = f"proximity:{longitude},{latitude}"


# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":500,
    "bias":bias,
    "apiKey": geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"
response=requests.get(base_url, params=params)
response_data = response.json()


In [110]:
print(response.url)

https://api.geoapify.com/v2/places?categories=accommodation&limit=500&bias=proximity%3A-74.006%2C40.7128&apiKey=be13e77afa4d4c778e3673d5943b0949


In [111]:
# Save JSON data to a file
with open('../Aumekubo/Resources/hotel.json', 'w') as json_file:
    json.dump(response_data, json_file, indent=4, sort_keys=True)

In [112]:
try:
    with open('../Aumekubo/hotel.json', 'w') as json_file:
        json.dump(response_data, json_file, indent=4, sort_keys=True)
    print("JSON file saved successfully.")
except Exception as e:
    print("Error occurred while saving JSON file:", str(e))


JSON file saved successfully.


In [113]:
# Specify the file path and name for the JavaScript output file
javascript_file_path = "hoteldata.js"

# Open the JavaScript file in write mode
javascript_file = open(javascript_file_path, "w")

# Convert the API response data to a JSON string
api_data = json.dumps(response_data)

# Write the JavaScript code to the file
javascript_file.write("var apiData = ")
javascript_file.write(api_data)
javascript_file.write(";")

# Close the file
javascript_file.close()

print(f"JavaScript output file saved at: {javascript_file_path}")

JavaScript output file saved at: hoteldata.js


In [114]:
# Print the json (pretty printed)
# print(json.dumps(response_data, indent=4, sort_keys=True))



In [115]:
response_data['features'][0]['properties']

{'name': 'The Beekman, A Thompson Hotel',
 'country': 'United States',
 'country_code': 'us',
 'state': 'New York',
 'county': 'New York County',
 'city': 'New York',
 'postcode': '10038',
 'district': 'Manhattan Community Board 1',
 'suburb': 'Manhattan',
 'street': 'Nassau Street',
 'housenumber': '123',
 'lon': -74.0069365,
 'lat': 40.7110329,
 'state_code': 'NY',
 'formatted': 'The Beekman, A Thompson Hotel, 123 Nassau Street, New York, NY 10038, United States of America',
 'address_line1': 'The Beekman, A Thompson Hotel',
 'address_line2': '123 Nassau Street, New York, NY 10038, United States of America',
 'categories': ['accommodation', 'accommodation.hotel', 'internet_access'],
 'details': ['details', 'details.facilities'],
 'datasource': {'sourcename': 'openstreetmap',
  'attribution': '© OpenStreetMap contributors',
  'license': 'Open Database Licence',
  'url': 'https://www.openstreetmap.org/copyright',
  'raw': {'name': 'The Beekman, A Thompson Hotel',
   'brand': 'Thompson 

In [116]:
data = []
for feature in response_data['features']:
    # Use the get method to handle missing keys
    hotel = feature['properties'].get('name', 'N/A')
    city = feature['properties'].get('city', 'N/A')
    postcode = feature['properties'].get('postcode', 'N/A')
    housenumber = feature['properties'].get('housenumber', 'N/A')
    street = feature['properties'].get('street', 'N/A')
    state = feature['properties'].get('state', 'N/A')
    address1 = feature['properties'].get('address_line1', 'N/A')
    address2 = feature['properties'].get('address_line2', 'N/A')
    data.append([hotel, city, postcode, housenumber, street,state, address1, address2])

# Create a DataFrame from the extracted data
df = pd.DataFrame(data, columns=["hotel", "city", "postcode","housenumber", "street", "state", "address1", "address2"])

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('hotels.csv', index=False)
df.to_excel('hotels.xlsx', index=False)


                                   hotel      city postcode housenumber  \
0          The Beekman, A Thompson Hotel  New York    10038         123   
1               Aloft Manhattan Downtown  New York    10038          49   
2                     Duane Street Hotel  New York    10013         182   
3                       Millenium Hilton  New York    10013          55   
4                    The Frederick Hotel  New York    10007         N/A   
..                                   ...       ...      ...         ...   
495                    Hilton Garden Inn  New York    11101       29-21   
496                                  N/A  New York    11101       40-30   
497  Wingate by Wyndham Long Island City  New York    11101         N/A   
498      Country Inn & Suites by Carlson  New York    11101       40-34   
499       Mayflower Howard Johnson Hotel  New York    11101         N/A   

              street     state                             address1  \
0      Nassau Street  New Yo

In [117]:
counter = 0

for place in response_data["features"]:
    print(f"hotel {counter+1}:")
    properties = place.get("properties", {})
    name = properties.get("name")
    city = properties.get("city")
    address_line1 = properties.get("address_line1")
    address_line2 = properties.get("address_line2")

    if name is not None:
        print(name)
    if city is not None:
        print(city)
    if address_line1 is not None:
        print(address_line1)
    if address_line2 is not None:
        print(address_line2)
        
    print("*"*20)
    counter = counter + 1
    if counter == 5:
        break


hotel 1:
The Beekman, A Thompson Hotel
New York
The Beekman, A Thompson Hotel
123 Nassau Street, New York, NY 10038, United States of America
********************
hotel 2:
Aloft Manhattan Downtown
New York
Aloft Manhattan Downtown
49 Ann Street, New York, NY 10038, United States of America
********************
hotel 3:
Duane Street Hotel
New York
Duane Street Hotel
182 Duane Street, New York, NY 10013, United States of America
********************
hotel 4:
Millenium Hilton
New York
Millenium Hilton
55 Church Street, New York, NY 10013, United States of America
********************
hotel 5:
The Frederick Hotel
New York
The Frederick Hotel
Chambers Street, New York, NY 10007, United States of America
********************


In [118]:
# Set the geographical coordinates for NYC
latitude = 40.7128
longitude = -74.0060

# Set the parameters for the type of place
categories = "entertainment"

# Set the parameters for the type of search
bias = f"proximity:{longitude},{latitude}"


# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":500,
    "bias":bias,
    "apiKey": geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"
response=requests.get(base_url, params=params)
response_ent = response.json()

In [119]:
# Print the json (pretty printed)
print(json.dumps(response_ent, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -74.01229239634087,
                    40.71265066401296
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Ronald O. Perelman Performing Arts Center",
                "address_line2": "58 Church Street, New York, NY 10048, United States of America",
                "categories": [
                    "building",
                    "building.entertainment",
                    "entertainment",
                    "entertainment.culture",
                    "entertainment.culture.theatre",
                    "wheelchair",
                    "wheelchair.yes"
                ],
                "city": "New York",
                "country": "United States",
                "country_code": "us",
                "county": "New York County",
                "datasource": {
                    "attribution

In [120]:
# Specify the file path and name for the JavaScript output file
javascript_file_path = "entdata.js"

# Open the JavaScript file in write mode
javascript_file = open(javascript_file_path, "w")

# Convert the API response data to a JSON string
api_data = json.dumps(response_ent)

# Write the JavaScript code to the file
javascript_file.write("var apiData = ")
javascript_file.write(api_data)
javascript_file.write(";")

# Close the file
javascript_file.close()

print(f"JavaScript output file saved at: {javascript_file_path}")

JavaScript output file saved at: entdata.js


In [121]:
# Save JSON data to a file
with open('../Aumekubo/Resources/entertainment.json', 'w') as json_file:
    json.dump(response_ent, json_file, indent=4, sort_keys=True)

In [122]:
try:
    with open('../Aumekubo/ent.json', 'w') as json_file:
        json.dump(response_ent, json_file, indent=4, sort_keys=True)
    print("JSON file saved successfully.")
except Exception as e:
    print("Error occurred while saving JSON file:", str(e))


JSON file saved successfully.


In [123]:
response_ent['features'][0]['properties']

{'name': 'Ronald O. Perelman Performing Arts Center',
 'country': 'United States',
 'country_code': 'us',
 'state': 'New York',
 'county': 'New York County',
 'city': 'New York',
 'postcode': '10048',
 'district': 'Manhattan Community Board 1',
 'suburb': 'Manhattan',
 'street': 'Church Street',
 'housenumber': '58',
 'lon': -74.01229284380794,
 'lat': 40.71261975,
 'state_code': 'NY',
 'formatted': 'Ronald O. Perelman Performing Arts Center, 58 Church Street, New York, NY 10048, United States of America',
 'address_line1': 'Ronald O. Perelman Performing Arts Center',
 'address_line2': '58 Church Street, New York, NY 10048, United States of America',
 'categories': ['building',
  'building.entertainment',
  'entertainment',
  'entertainment.culture',
  'entertainment.culture.theatre',
  'wheelchair',
  'wheelchair.yes'],
 'details': ['details',
  'details.building',
  'details.contact',
  'details.facilities',
  'details.wiki_and_media'],
 'datasource': {'sourcename': 'openstreetmap',


In [124]:
data_ent = []

for feature in response_ent['features']:
    # Use the get method to handle missing keys
    entertainment = feature['properties'].get('name', 'N/A')
    city = feature['properties'].get('city', 'N/A')
    postcode = feature['properties'].get('postcode', 'N/A')
    housenumber = feature['properties'].get('housenumber', 'N/A')
    street = feature['properties'].get('street', 'N/A')
    state = feature['properties'].get('state', 'N/A')
    address1 = feature['properties'].get('address_line1', 'N/A')
    address2 = feature['properties'].get('address_line2', 'N/A')
    data_ent.append([entertainment, city, postcode, housenumber, street,state, address1, address2])

# Create a DataFrame from the extracted data
ent_df = pd.DataFrame(data_ent, columns=["entertainment", "city", "postcode","housenumber", "street", "state", "address1", "address2"])

# Display the DataFrame
print(ent_df)

# Save the DataFrame to a CSV file
ent_df.to_csv('entertainment2.csv', index=False)
ent_df.to_excel('entertainment2.xlsx', index=False)


                                 entertainment      city postcode housenumber  \
0    Ronald O. Perelman Performing Arts Center  New York    10048          58   
1                           81 Leonard Gallery  New York    10013         N/A   
2                       9/11 Memorial & Museum  New York    10007         180   
3                                     Mmuseumm  New York    10013           4   
4                               The Print Room  New York    10013          89   
..                                         ...       ...      ...         ...   
495                             Dorsky Gallery  New York    11101       11-03   
496                          Prospect Park Zoo  New York    11225         450   
497                                Zankel Hall  New York    10019         N/A   
498                           Amos Eno Gallery  New York    11206          56   
499             Syndicated Bar Theater Kitchen  New York    11206          40   

               street     s

In [125]:
# Set the geographical coordinates for NYC
latitude = 40.7128
longitude = -74.0060

# Set the parameters for the type of place
categories = "public_transport"

# Set the parameters for the type of search
bias = f"proximity:{longitude},{latitude}"


# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":500,
    "bias":bias,
    "apiKey": geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"
response=requests.get(base_url, params=params)
response_transport = response.json()

In [126]:
# Print the json (pretty printed)
print(json.dumps(response_transport, indent=4, sort_keys=True))


{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -74.00507019999999,
                    40.71235000035877
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Brooklyn Bridge\u2013City Hall",
                "address_line2": "Centre Street, New York, NY 10000, United States of America",
                "categories": [
                    "public_transport",
                    "public_transport.subway",
                    "wheelchair",
                    "wheelchair.yes"
                ],
                "city": "New York",
                "country": "United States",
                "country_code": "us",
                "county": "New York County",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
               

In [127]:
# Specify the file path and name for the JavaScript output file
javascript_file_path = "transpdata.js"

# Open the JavaScript file in write mode
javascript_file = open(javascript_file_path, "w")

# Convert the API response data to a JSON string
api_data = json.dumps(response_transport)

# Write the JavaScript code to the file
javascript_file.write("var apiData = ")
javascript_file.write(api_data)
javascript_file.write(";")

# Close the file
javascript_file.close()

print(f"JavaScript output file saved at: {javascript_file_path}")

JavaScript output file saved at: transpdata.js


In [128]:
# Save JSON data to a file
with open('../Aumekubo/Resources/transport.json', 'w') as json_file:
    json.dump(response_transport, json_file, indent=4, sort_keys=True)
    

In [129]:
try:
    with open('../Aumekubo/transport.json', 'w') as json_file:
        json.dump(response_transport, json_file, indent=4, sort_keys=True)
    print("JSON file saved successfully.")
except Exception as e:
    print("Error occurred while saving JSON file:", str(e))


JSON file saved successfully.


In [130]:
response_transport['features'][0]['properties']

{'name': 'Brooklyn Bridge–City Hall',
 'country': 'United States',
 'country_code': 'us',
 'state': 'New York',
 'county': 'New York County',
 'city': 'New York',
 'postcode': '10000',
 'district': 'Manhattan Community Board 1',
 'suburb': 'Manhattan',
 'quarter': 'Lower Manhattan',
 'street': 'Centre Street',
 'lon': -74.0050702,
 'lat': 40.71235,
 'state_code': 'NY',
 'formatted': 'Brooklyn Bridge–City Hall, Centre Street, New York, NY 10000, United States of America',
 'address_line1': 'Brooklyn Bridge–City Hall',
 'address_line2': 'Centre Street, New York, NY 10000, United States of America',
 'categories': ['public_transport',
  'public_transport.subway',
  'wheelchair',
  'wheelchair.yes'],
 'details': ['details', 'details.facilities', 'details.wiki_and_media'],
 'datasource': {'sourcename': 'openstreetmap',
  'attribution': '© OpenStreetMap contributors',
  'license': 'Open Database Licence',
  'url': 'https://www.openstreetmap.org/copyright',
  'raw': {'name': 'Brooklyn Bridge–

In [131]:
data_trans = []

for feature in response_transport['features']:
    # Use the get method to handle missing keys
    tranportation = feature['properties'].get('name', 'N/A')
    city = feature['properties'].get('city', 'N/A')
    postcode = feature['properties'].get('postcode', 'N/A')
    housenumber = feature['properties'].get('housenumber', 'N/A')
    street = feature['properties'].get('street', 'N/A')
    state = feature['properties'].get('state', 'N/A')
    address1 = feature['properties'].get('address_line1', 'N/A')
    address2 = feature['properties'].get('address_line2', 'N/A')
    data_trans.append([tranportation, city, postcode, housenumber, street,state, address1, address2])

# Create a DataFrame from the extracted data
trans_df = pd.DataFrame(data_trans, columns=["tranportation", "city", "postcode","housenumber", "street", "state", "address1", "address2"])

# Display the DataFrame
print(trans_df)

# Save the DataFrame to a CSV file
trans_df.to_csv('transport3.csv', index=False)
trans_df.to_excel('transport3.xlsx', index=False)


                                  tranportation         city postcode  \
0                     Brooklyn Bridge–City Hall     New York    10000   
1                                     City Hall     New York    10003   
2                      Park Row & Spruce Street     New York    10007   
3                      Park Row & Spruce Street     New York    10000   
4                    Chambers Street & Broadway     New York    10000   
..                                          ...          ...      ...   
495               6th Avenue & West 18th Street     New York    10011   
496  Grand Street at Luis Munoz Marin Boulevard  Jersey City    07304   
497                                Grove Street  Jersey City    07302   
498         Flushing Avenue & Cumberland Street     New York    11237   
499                         Newport Centre Mall  Jersey City    07310   

    housenumber                      street       state  \
0           N/A               Centre Street    New York   
1    